### CNN Model Testing

#### Imports

In [ ]:
import tensorflow as tf
import pandas as pd
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score

#### Load Dataset

In [ ]:
# Load and preprocess images
def load_image(image_name, label):
    image_path = tf.strings.join([image_folder, '/', image_name])
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [224, 224])  # Resize to match the input shape of the CNN
    image = image / 255.0  # Normalize pixel values to [0, 1]
    return image, label

# Create a TensorFlow dataset from a DataFrame
def create_tf_dataset(dataframe, image_folder, batch_size=32):
    image_names = dataframe['image'].values
    labels = dataframe['MEL'].values
    dataset = tf.data.Dataset.from_tensor_slices((image_names, labels))
    dataset = dataset.map(load_image, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.shuffle(buffer_size=1000).batch(batch_size).prefetch(buffer_size=tf.data.AUTOTUNE)
    return dataset

csv_path = 'preprocessed_dataset/ISIC_2019_Test_Balanced.csv'
image_folder = 'preprocessed_dataset'

# Load the CSV file
df = pd.read_csv(csv_path)
df['image'] = df['image'] + '.jpg'
print(f"Loaded {len(df)} images from {csv_path}")

# Create TensorFlow dataset
dataset = create_tf_dataset(df, image_folder)

print("Dataset loaded and preprocessed.")

#### Preview Dataset

In [ ]:
# Preview a batch of images and labels
def preview_dataset(dataset, num_images=5):
    for images, labels in dataset.take(1):  # Take one batch
        plt.figure(figsize=(15, 15))
        for i in range(num_images):
            ax = plt.subplot(1, num_images, i + 1)
            plt.imshow(images[i].numpy())
            plt.title(f"Label: {int(labels[i].numpy())}")
            plt.axis("off")
        plt.show()

# Preview the testing dataset
print("Previewing testing dataset:")
preview_dataset(dataset)

#### Import model

In [ ]:
model_path = '../model_training/best_model.h5'
model = load_model(model_path)

model.summary()

#### Evaluate CNN model

In [ ]:
y_true = []
y_pred = []

for images, labels in dataset:
    y_true.extend(labels.numpy())  # True labels
    predictions = model.predict(images)  # Predicted probabilities
    y_pred.extend((predictions > 0.5).astype(int).flatten())  # Convert probabilities to binary predictions

# Calculate metrics
conf_matrix = confusion_matrix(y_true, y_pred)
accuracy = accuracy_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)

# Print results
print("Confusion Matrix:")
print(conf_matrix)
print(f"Accuracy: {accuracy:.4f}")
print(f"F1 Score: {f1:.4f}")